# Dev mp-client (python3 host)

In [4]:
import socket, os, time

code = [
    "print('empty string follows')",
    "",
    "print(5**9)",
    "print(2**23)",
"""
uid = bytes(6)
try:
    import machine
    uid = machine.unique_id()
except:
    import microcontroller
    uid = microcontroller.cpu.uid
print(":".join("{:02x}".format(x) for x in uid), end="")
""",
"""
for i in range(5):
    print(i, i**5)
""",
"""
print('a nasty error')
5/0
""",
    "'back to nice'",
]

VERSION = 'ns-V1'
PWD = '#$&*(98a7D$8{9hS'

# mp://10.39.40.129:54321
ADDR = ('10.39.40.129', 54321)

EOT = b'\x04'

class MpDevice:
    
    def __init__(self, addr):
        self.addr = addr
        self.sock = ...
        
    def recv_exactly(self, sz:int):
        # read sz bytes from socket
        data = b''
        while len(data) < sz:
            b = self.sock.recv(sz)
            if b == b'':
                raise Exception("connection closed")
            data += b
        # print(f"_recv_exactly [{sz}] {data}")
        return data
    
    def readline(self, timeout=1):
        """Blocking"""
        res = b''
        start = time.monotonic()
        # let's give the MCU some time ...
        while (time.monotonic() - start) < timeout:
            try:
                b = self.sock.recv(1)
            except socket.timeout:
                time.sleep(0.1)
                continue
            if b == b'':
                raise Exception("connection closed")
            if b == b'\n':
                return res.decode()
            res += b
        return res.decode()

    def connect(self):
        self.sock = s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect(self.addr)
        s.settimeout(1)
        print("connected")
        version = self.readline()
        assert version == VERSION, "Version mismatch"
        s.sendall(PWD.encode())
        s.sendall(b'\n')
        ok = self.readline()
        if ok != 'OK':
            raise Exception(ok)
        return self
    
    def __enter__(self):
        return self

    def __exit__(self, *args):
        self.sock.sendall(b'bye\n')
        try:
            self.sock.close()
        except:
            pass
        self.sock = None
        
    def do_eval_exec(self, code):
        if len(code) == 0: return
        if isinstance(code, str): code = code.encode()
        sock = self.sock
        sock.sendall(b'evex\n')
        sock.sendall(str(len(code)).encode())
        sock.sendall(b'\n')        
        sock.sendall(code)
        # compilation may take a long time!
        ok = self.readline(60)
        assert ok == 'OK', f"server not acknowledging receipt of code: {ok}"
        result = b""
        print("-->")
        while result.count(EOT) < 2:
            try:
                res = sock.recv(2048)
            except socket.timeout:
                time.sleep(0.1)
                continue
            print(res.decode())
            if res == b'': raise Exception("closed")
            result += res
        ans, err, _ = result.split(EOT)

    def do_fget(self, src, dst):
        sock = self.sock
        sock.sendall(b'fget\n')
        sock.sendall(src.encode())
        sock.sendall(b'\n')
        ok = self.readline()
        assert ok == 'OK', ok
        sz = int(self.readline())
        n = 0
        with open(dst, 'wb') as f:
            n = 0
            while n < sz:
                b = sock.recv(min(1024, sz-n))
                f.write(b)
                n += len(b)

    def do_fput(self, src, dst):
        sz = os.path.getsize(src)
        sock = self.sock
        sock.sendall(b"fput\n")
        sock.sendall(dst.encode())
        sock.sendall(b'\n')
        sock.sendall(str(sz).encode())
        sock.sendall(b'\n')
        with open(src, 'rb') as f:
            x = f.read(sz)
            sock.sendall(x)
        ok = self.readline(timeout=10+0.01*sz)
        assert ok == 'OK', ok
                
    def do_echo(self, data):
        if isinstance(data, str):
            data = data.encode()
        sock = self.sock
        sock.sendall(b"echo\n")
        sock.sendall(str(len(data)).encode())
        sock.sendall(b"\n")
        sock.sendall(data)
        result = b""
        while len(result) < len(data):
            res = sock.recv(1024)
            if len(data) == 0: break
            result += res
        assert result == data
        
        
SRC = '/Users/boser/Dropbox/Documents/Projects/IoT/iot49/airlift/mcu/lib/socket.py'

        
with MpDevice(ADDR) as client:
    client.connect()
    # client.do_eval_exec("import machine; machine.reset()")
    if True:
        for c in code:
            client.do_eval_exec(c)
    if True:
        client.do_echo(b'abc')
        # client.do_echo(bytes(os.urandom(20000)))
    if True:
        print("fput ...")
        client.do_fput(SRC, 'dummy.txt')
        client.do_fget('dummy.txt', 'delete_me.txt')
    if True:
        with open(SRC) as f:
            c1 = f.read()
            # print("C1", c1)
        with open('delete_me.txt') as f:
            # print("\n", '-'*50)
            c2 = f.read()
            # print("C2", c2)
        if c1 == c2:
            print("match!")
        else:
            print(f"{c1}\n\n!=\n\n{c2}")
        code = c1*20 + '\nprint("Got it!")\n'
        print('len(code)', len(code))
        client.do_eval_exec(code)
    
!rm delete_me.txt



connected
-->
empty string follows


-->
1953125



-->
8388608



-->
27:00:55:00:09:50:52:42:4e:30:39:20

-->
0
 0
1 1

2 
32

3 243

4 1024

-->
a nasty error


Traceback (most recent call last):
  File "/flash/lib/mp/server.py", line 105, in 
do_evex

  File "<string>", line 3, in <module>
ZeroDivisionError: divide by zero

-->
'back to nice'

fput ...
match!
len(code) 8258
-->
Got it!


